# 기업은행 뉴스 스크래핑
이 노트북은 '기업은행'이 포함된 뉴스를 수집합니다.

In [1]:
# 필요한 라이브러리 설치 (처음 한 번만 실행)
# !python -m pip install requests beautifulsoup4 pandas lxml feedparser

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import feedparser
import urllib.parse

In [3]:
def search_google_news(keyword, display=10):
    """
    Google News RSS에서 뉴스 수집
    keyword: 검색 키워드
    display: 가져올 뉴스 개수
    """
    news_list = []
    
    try:
        encoded_keyword = urllib.parse.quote(keyword)
        rss_url = f"https://news.google.com/rss/search?q={encoded_keyword}&hl=ko&gl=KR&ceid=KR:ko"
        
        print(f"[Google News] RSS URL: {rss_url}")
        print("뉴스를 가져오는 중...")
        
        feed = feedparser.parse(rss_url)
        
        if not feed.entries:
            print("[Google News] RSS에서 뉴스를 찾을 수 없습니다.")
            return []
        
        print(f"✓ [Google News] {len(feed.entries)}개의 뉴스를 찾았습니다!")
        
        for idx, entry in enumerate(feed.entries[:display]):
            try:
                title = entry.get('title', '제목 없음')
                link = entry.get('link', '')
                pub_date = entry.get('published', '날짜 정보 없음')
                summary = entry.get('summary', '요약 없음')
                source = entry.get('source', {}).get('title', '언론사 정보 없음')
                
                if summary and summary != '요약 없음':
                    summary_soup = BeautifulSoup(summary, 'html.parser')
                    summary = summary_soup.get_text(strip=True)
                
                news_list.append({
                    '번호': idx + 1,
                    '제목': title,
                    '언론사': source,
                    '날짜': pub_date,
                    '요약': summary[:200] if len(summary) > 200 else summary,
                    '링크': link,
                    '출처': 'Google News'
                })
            except Exception as e:
                print(f"항목 {idx + 1} 파싱 오류: {e}")
                continue
        
        return news_list
    
    except Exception as e:
        print(f"[Google News] 뉴스 수집 오류: {e}")
        import traceback
        traceback.print_exc()
        return []


def search_naver_news(keyword, display=10):
    """
    네이버 뉴스 검색 (모바일 웹 스크래핑)
    keyword: 검색 키워드
    display: 가져올 뉴스 개수
    """
    news_list = []
    
    try:
        encoded_keyword = urllib.parse.quote(keyword)
        # 모바일 네이버 뉴스 검색 URL (날짜순 정렬)
        url = f"https://m.search.naver.com/search.naver?where=m_news&query={encoded_keyword}&sort=1"
        
        print(f"[네이버 뉴스] URL: {url}")
        print("뉴스를 가져오는 중...")
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1'
        }
        
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 모바일 네이버 뉴스 검색 결과 파싱 (FDS 디자인 시스템)
        news_container = soup.select_one('div.fds-news-item-list-tab')
        
        if not news_container:
            print("[네이버 뉴스] 뉴스 컨테이너를 찾을 수 없습니다.")
            return []
        
        # 뉴스 아이템들 - 직접 자식만 선택 (중첩 방지)
        news_items = news_container.find_all('div', class_='sds-comps-vertical-layout', recursive=False)
        
        if not news_items:
            print("[네이버 뉴스] 뉴스를 찾을 수 없습니다.")
            return []
        
        print(f"✓ [네이버 뉴스] {len(news_items)}개의 뉴스를 찾았습니다!")
        
        seen_links = set()  # 중복 링크 체크용
        count = 0
        
        for item in news_items:
            if count >= display:
                break
            
            try:
                # 뉴스 링크들 찾기
                links = item.find_all('a', href=True)
                
                title = None
                link = None
                summary = None
                source = None
                pub_date = None
                
                for a_tag in links:
                    href = a_tag.get('href', '')
                    text = a_tag.get_text(strip=True)
                    
                    # Keep 버튼, 빈 링크 제외
                    if 'keep.naver.com' in href or href == '#' or not text:
                        continue
                    
                    # 뉴스 링크 찾기 (20자 이상의 텍스트)
                    if ('news.naver.com' in href or href.startswith('http')) and len(text) > 20:
                        if title is None:
                            title = text
                            link = href
                        elif summary is None and text != title:
                            summary = text
                
                # 제목이나 링크가 없거나 중복이면 건너뛰기
                if not title or not link or link in seen_links:
                    continue
                
                seen_links.add(link)
                
                # span 태그에서 언론사, 날짜 추출
                spans = item.find_all('span')
                seen_texts = set()
                
                for span in spans:
                    text = span.get_text(strip=True)
                    if not text or text in seen_texts or len(text) > 30:
                        continue
                    seen_texts.add(text)
                    
                    # 시간 패턴 (X분 전, X시간 전, X일 전)
                    if '전' in text and any(c.isdigit() for c in text):
                        if pub_date is None:
                            pub_date = text
                    # 언론사 (시간이 아니고, 제목과 다른 짧은 텍스트)
                    elif source is None and len(text) > 1 and text not in title:
                        source = text
                
                news_list.append({
                    '번호': count + 1,
                    '제목': title,
                    '언론사': source or '언론사 정보 없음',
                    '날짜': pub_date or '날짜 정보 없음',
                    '요약': (summary[:200] if summary and len(summary) > 200 else summary) or '요약 없음',
                    '링크': link,
                    '출처': '네이버 뉴스'
                })
                count += 1
                
            except Exception as e:
                print(f"항목 파싱 오류: {e}")
                continue
        
        return news_list
    
    except Exception as e:
        print(f"[네이버 뉴스] 뉴스 수집 오류: {e}")
        import traceback
        traceback.print_exc()
        return []


def search_all_news(keyword, display_per_source=10):
    """
    모든 소스(Google News, 네이버)에서 뉴스 수집
    keyword: 검색 키워드
    display_per_source: 각 소스별 가져올 뉴스 개수
    """
    all_news = []
    
    print("=" * 60)
    print(f"'{keyword}' 키워드로 뉴스를 검색합니다...")
    print("=" * 60)
    
    # Google News 검색
    print("\n[1/2] Google News 검색 중...")
    google_news = search_google_news(keyword, display_per_source)
    all_news.extend(google_news)
    
    time.sleep(1)  # 요청 간격 조절
    
    # 네이버 뉴스 검색
    print("\n[2/2] 네이버 뉴스 검색 중...")
    naver_news = search_naver_news(keyword, display_per_source)
    all_news.extend(naver_news)
    
    # 번호 재정렬
    for idx, news in enumerate(all_news):
        news['번호'] = idx + 1
    
    print("\n" + "=" * 60)
    print(f"총 {len(all_news)}개의 뉴스 수집 완료!")
    print(f"  - Google News: {len(google_news)}개")
    print(f"  - 네이버 뉴스: {len(naver_news)}개")
    print("=" * 60)
    
    return all_news

In [4]:
# 기업은행 은행장 뉴스 검색 (Google + 네이버)
print("'기업은행 은행장' 뉴스를 검색합니다...\n")
news_data = search_all_news("기업은행 은행장", display_per_source=10)

if news_data:
    print(f"\n총 {len(news_data)}개의 뉴스를 찾았습니다.\n")
    
    # 데이터프레임으로 변환
    df = pd.DataFrame(news_data)
    
    # 결과 출력
    print("=" * 80)
    for idx, row in df.iterrows():
        print(f"\n[{row['번호']}] [{row['출처']}] {row['제목']}")
        print(f"언론사: {row['언론사']} | 날짜: {row['날짜']}")
        print(f"요약: {row['요약'][:100]}...")
        print(f"링크: {row['링크']}")
        print("-" * 80)
else:
    print("뉴스를 찾을 수 없습니다.")

'기업은행 은행장' 뉴스를 검색합니다...

'기업은행 은행장' 키워드로 뉴스를 검색합니다...

[1/2] Google News 검색 중...
[Google News] RSS URL: https://news.google.com/rss/search?q=%EA%B8%B0%EC%97%85%EC%9D%80%ED%96%89%20%EC%9D%80%ED%96%89%EC%9E%A5&hl=ko&gl=KR&ceid=KR:ko
뉴스를 가져오는 중...
✓ [Google News] 100개의 뉴스를 찾았습니다!

[2/2] 네이버 뉴스 검색 중...
[네이버 뉴스] URL: https://m.search.naver.com/search.naver?where=m_news&query=%EA%B8%B0%EC%97%85%EC%9D%80%ED%96%89%20%EC%9D%80%ED%96%89%EC%9E%A5&sort=1
뉴스를 가져오는 중...
✓ [네이버 뉴스] 15개의 뉴스를 찾았습니다!

총 20개의 뉴스 수집 완료!
  - Google News: 10개
  - 네이버 뉴스: 10개

총 20개의 뉴스를 찾았습니다.


[1] [Google News] 기업은행, 생산적 금융 발맞춘 조직개편 변화 주목 - fetv.co.kr
언론사: fetv.co.kr | 날짜: Sun, 18 Jan 2026 23:00:19 GMT
요약: 기업은행, 생산적 금융 발맞춘 조직개편 변화 주목fetv.co.kr...
링크: https://news.google.com/rss/articles/CBMiX0FVX3lxTE01SzZ1NkxSaXNxM2U2bFdSU3JET1Z2OW5kWUpiZDJSY2tEd0Q2dkJzYXVzYktSQk5MZjdkb2NybVdrRnN6VkpGbWxjR29hXzczRnZIS2xLdlZTU0RrOXRV?oc=5
--------------------------------------------------------------------------------

[2] [Google News] 기

In [5]:
# 데이터프레임 확인
if news_data:
    df

In [6]:
# CSV 파일로 저장
if news_data:
    filename = f"기업은행_뉴스_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n뉴스 데이터를 '{filename}' 파일로 저장했습니다.")


뉴스 데이터를 '기업은행_뉴스_20260119_092218.csv' 파일로 저장했습니다.


In [7]:
# 추가: 특정 키워드로 필터링
if news_data:
    search_keyword = input("추가로 필터링할 키워드를 입력하세요 (Enter로 건너뛰기): ")
    
    if search_keyword:
        filtered_df = df[df['제목'].str.contains(search_keyword) | df['요약'].str.contains(search_keyword)]
        print(f"\n'{search_keyword}' 키워드로 필터링된 결과: {len(filtered_df)}개")
        filtered_df


'은행장' 키워드로 필터링된 결과: 14개
